In [1]:
import pandas as pd
import math as mt
import numpy as np
import os

# Selecting and Merging

We will work with a database consisting in 9 tables. So our first job will be to select and merge the relevant information. Of course, keeping in mind our goal: to estimate the freight price of the order.

In [2]:
#Taking a quick look at our data

all_files = os.listdir('olist_database_files/')
olist_files = [i for i in all_files if re.match('olist_',i)] #all olist csv files starts with 'olist_'

for file in olist_files:
    provisory_df = pd.read_csv('olist_database_files/' + file)
    print(f'\nfile name: {file}',
          f'\nfile shape: {provisory_df.shape}')
    display(provisory_df.sample(2))


file name: olist_customers_dataset.csv 
file shape: (99441, 5)


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
65487,77ed5ae0acf4f7fd1298ef46f09eab67,0ad16bdc7651e5ac483666ab0215bf93,4890,sao paulo,SP
14194,45ff99298b13ff934ceac68fe8ce405f,24f2709edd3033672c85489a9d01718d,2408,sao paulo,SP



file name: olist_geolocation_dataset.csv 
file shape: (1000163, 5)


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
680605,42721,-12.888215,-38.339252,lauro de freitas,BA
872128,85010,-25.392973,-51.464498,guarapuava,PR



file name: olist_orders_dataset.csv 
file shape: (99441, 8)


...

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
88221,7ab37db15e97dfc0af086a219316d803,8f01c37e40b9262e477bb2e2d01058b1,5,NaN,Produto de ótima qualidade!,2017-03-11 00:00:00,2017-03-12 22:43:14
92535,11465cf4b8e985fde6625c294f34f2c9,1b4f84742c7c3d03f7c6d519212514ab,4,NaN,Chegou rápido e tudo em ordem!,2017-02-15 00:00:00,2017-02-15 18:42:37



file name: olist_products_dataset.csv 
file shape: (32951, 9)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
21291,3de4498dcdded5abe7349bd43c401c1f,relogios_presentes,60.0,418.0,9.0,300.0,16.0,2.0,11.0
27099,ecbe401f10ecd3b822d940d63cc3311e,eletronicos,60.0,915.0,4.0,700.0,16.0,7.0,12.0



file name: olist_sellers_dataset.csv 
file shape: (3095, 4)


,seller_id,seller_zip_code_prefix,seller_city,seller_state
51,d71d863e5ef30d94e440c11be17dcd8f,6144,osasco,SP
1070,851773c885feb5e2da3b4f82bc2b17ce,18683,lencois paulista,SP


First of all, we note that the freight price is addressed in the table "olist_order_items_dataset" and the price is given by item, not by order. We also notice that some tables are useless for our purpose, especifically the tables "olist_order_reviews_dataset" and olist_order_payments_dataset have no relevant infos. All the others Tables have informations that may be useful. We will list here the columns and keys that worth retrieving from each table: 

+ olist_customers_dataset -> Keys:['customer_id','customer_zip_code_prefix'] || Data: ['customer_state'] 
+ olist_geolocation_dataset -> Keys:['geolocation_zip_code_prefix'] || Data: ['geolocation_lat','geolocation_lng']
+ olist_orders_dataset -> Keys:['order_id','customer_id'] || Data: ['order_status']           
+ olist_order_items_dataset -> Keys:['order_id','product_id','seller_id'] || Data: ['price','freight_value']
+ olist_products_dataset -> Keys:['product_id'] || Data: ['product_weight_g','product_length_cm','product_height_cm','product_width_cm']
+ olist_sellers_dataset -> Keys:['seller_id'] || Data: ['seller_zip_code_prefix','seller_state']


Now that we defined what data we want to keep, we just need to load and merge tables to create our raw DataFrame

In [3]:
# lets start with our main table:

olist_order_items_dataset = pd.read_csv('olist_database_files/olist_order_items_dataset.csv')
olist_order_items_dataset.drop('order_item_id',axis =1,inplace = True)

print(f'Shape: {olist_order_items_dataset.shape}')
olist_order_items_dataset.head(1)

Shape: (112650, 6)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29


In [4]:
#merging with olist_orders_dataset

olist_orders_dataset = pd.read_csv('olist_database_files/olist_orders_dataset.csv')
olist_orders_dataset = olist_orders_dataset.loc[:,['order_id','customer_id','order_status']]

merged_df = pd.merge(olist_order_items_dataset,olist_orders_dataset, on ='order_id')
print(f'Shape: {merged_df.shape}')
merged_df.head(1)

Shape: (112650, 8)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered


In [5]:
#merging with olist_customers_dataset

olist_products_dataset = pd.read_csv('olist_database_files/olist_products_dataset.csv')
olist_products_dataset2 = olist_products_dataset.loc[:,['product_id','product_weight_g','product_length_cm',
                                                       'product_height_cm','product_width_cm']]

merged_df_v1 = pd.merge(merged_df,olist_products_dataset2, on ='product_id')
print(f'Shape: {merged_df_v1.shape}')
merged_df_v1.head(1)

Shape: (112650, 12)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,650.0,28.0,9.0,14.0


In [6]:
#merging with olist_customers_dataset

olist_customers_dataset = pd.read_csv("olist_database_files/olist_customers_dataset.csv")
olist_customers_dataset = olist_customers_dataset.loc[:,['customer_id','customer_zip_code_prefix','customer_state']]

merged_df_v2 = pd.merge(merged_df_v1,olist_customers_dataset,on='customer_id')
print(f'Shape: {merged_df_v2.shape}')
merged_df_v2.head(1)

Shape: (112650, 14)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,650.0,28.0,9.0,14.0,28013,RJ


In [7]:
#merging with olist_sellers_dataset

olist_sellers_dataset = pd.read_csv("olist_database_files/olist_sellers_dataset.csv")
olist_sellers_dataset = olist_sellers_dataset.loc[:,['seller_id','seller_zip_code_prefix','seller_state']]

merged_df_v3 = pd.merge(merged_df_v2,olist_sellers_dataset,on='seller_id')
print(f'Shape: {merged_df_v3.shape}')
merged_df_v3.head(1)

Shape: (112650, 16)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,650.0,28.0,9.0,14.0,28013,RJ,27277,SP


In [8]:
#merging with olist_geolocation_dataset

olist_geolocation_dataset = pd.read_csv('olist_database_files/olist_geolocation_dataset.csv')
olist_geolocation_dataset = olist_geolocation_dataset.loc[:,['geolocation_zip_code_prefix',
                                                             'geolocation_lat',
                                                             'geolocation_lng']]
display(olist_geolocation_dataset.head(1))

#The geolocation dataset will need special tratement, because we have diferent latitudes and longitudes for the same zip code

print(f'Row numbers: {len(olist_geolocation_dataset)}'
      f'\nUnique Zip Codes: {olist_geolocation_dataset.geolocation_zip_code_prefix.nunique()}')

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,1037,-23.545621,-46.639292


Row numbers: 1000163
Unique Zip Codes: 19015


Como???????

Lets test an hypothesis: 

For the same Zip Codes, the geolocation will be very close. So it seens fair to just use the mean value

+ H0 = geolocation doesn't depend on zip code
+ Ha = geolocation is very similar for the same zip code


In [9]:
#grouping the geolation by zip code

olist_geolocation_grouped = olist_geolocation_dataset.groupby('geolocation_zip_code_prefix',as_index = False).mean()
olist_geolocation_grouped.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,1001,-23.550190,-46.634024
1,1002,-23.548146,-46.634979
2,1003,-23.548994,-46.635731


In [10]:
#for the geolocation we will need to merge two times, we have one geolocation for the seller and one for the customer

#merging with customer

merged_df_v4 = pd.merge(merged_df_v3,
                        olist_geolocation_grouped,
                        left_on ='customer_zip_code_prefix',
                        right_on = 'geolocation_zip_code_prefix',
                        how = 'left')

merged_df_v4.drop('geolocation_zip_code_prefix',axis =1,inplace = True)
#lets rename the geolocation columns to specify that it concern to the customer 
merged_df_v4.rename(columns ={'geolocation_lat':'customer_lat','geolocation_lng':'customer_lng'},inplace = True)

print(f'Shape: {merged_df_v4.shape}')
merged_df_v4.head(1)

Shape: (112650, 18)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state,customer_lat,customer_lng
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,650.0,28.0,9.0,14.0,28013,RJ,27277,SP,-21.762775,-41.309633


In [11]:
#merging geolocation with seller

merged_df_v5 = pd.merge(merged_df_v4,
                        olist_geolocation_grouped,
                        left_on ='seller_zip_code_prefix',
                        right_on = 'geolocation_zip_code_prefix',
                        how = 'left')

merged_df_v5.drop('geolocation_zip_code_prefix',axis =1,inplace = True)

#lets rename the geolocation columns to specify that it concern to the customer 
merged_df_v5.rename(columns ={'geolocation_lat':'seller_lat','geolocation_lng':'seller_lng'},inplace = True)

print(f'Shape: {merged_df_v5.shape}')
merged_df_v5.head(1)

Shape: (112650, 20)


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state,customer_lat,customer_lng,seller_lat,seller_lng
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,650.0,28.0,9.0,14.0,28013,RJ,27277,SP,-21.762775,-41.309633,-22.496953,-44.127492


We have finally merged all the tables and get all the infos that may be relevant! let take a look at our DataFrame

In [12]:
raw_freight_df = merged_df_v5.copy()

print(raw_freight_df.columns)
raw_freight_df.sample(3)

Index(['order_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value', 'customer_id', 'order_status', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_zip_code_prefix', 'customer_state', 'seller_zip_code_prefix',
       'seller_state', 'customer_lat', 'customer_lng', 'seller_lat',
       'seller_lng'],
      dtype='object')


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state,customer_lat,customer_lng,seller_lat,seller_lng
74529,30acc4fc5f485f20a89c9af9b0b67481,82f03c68ad0f694bab78643d0ff8c67f,a3a38f4affed601eb87a97788c949667,2017-08-07 17:05:14,129.99,16.67,8366ba08152f16bb9b8534350e085166,delivered,671.0,33.0,15.0,28.0,13468,SP,89204,SC,-22.754938,-47.349404,-26.287307,-48.851802
76172,f09e65822983a5488ca34c053ed555ac,3d5837f86205fe83f03fb5f7e4d5b9cf,9d4db00d65d7760644ac0c14edb5fd86,2018-07-09 09:31:45,109.80,10.08,104b1c416aeb72edd6c2e03d513c72d9,delivered,1017.0,37.0,5.0,37.0,5525,SP,18025,SP,-23.601800,-46.751801,-23.515612,-47.443461
65670,e985dab0862fa067f40e4c0d54a7405f,48abab76a7346e27ed907fed3f388483,ca3bd7cd9f149df75950150d010fe4a2,2017-03-31 17:27:44,34.20,14.12,384d79f1020310fdeecaf440abd4f0ca,delivered,4900.0,25.0,60.0,25.0,9636,SP,4361,SP,-23.652227,-46.574441,-23.643527,-46.662809


Now that our DataFrame is merged lets do some general checking and treatment

# Treating Data

In [13]:
raw_freight_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   order_id                  112650 non-null  object 
 1   product_id                112650 non-null  object 
 2   seller_id                 112650 non-null  object 
 3   shipping_limit_date       112650 non-null  object 
 4   price                     112650 non-null  float64
 5   freight_value             112650 non-null  float64
 6   customer_id               112650 non-null  object 
 7   order_status              112650 non-null  object 
 8   product_weight_g          112632 non-null  float64
 9   product_length_cm         112632 non-null  float64
 10  product_height_cm         112632 non-null  float64
 11  product_width_cm          112632 non-null  float64
 12  customer_zip_code_prefix  112650 non-null  int64  
 13  customer_state            112650 non-null  o

We can see that we have missing data.
+ 18 items without infos about weight and measurements
+ 302 items without customer coordinates
+ 253 items without seller coordinates

Lets start treating the coordinates problem

The problem with the missing coordinates is that a few times, the customer or the seller zip codes were not found in the olist_geolocation_dataset. 

We know that the zip codes are ordered by distance, for example, zip code 1037 = coordinates(-23.545621,-46.639292) while zip code 1046 = coordinates(-23.546081,-46.644820). This means that we can fill those missing coordinates with the coordinate of the closest zip code, without losing much. 

In [14]:
#findind the closest zip code of missing zip codes and filling missing coordinates

def find_closest_coordinate(df):
    
    #fixing customer coordinates
    if mt.isnan(df.customer_lat):
        customer_zip = df.customer_zip_code_prefix
        zip_final_diff = float('inf')
        for zip_code,lat,lng in zip(olist_geolocation_grouped.geolocation_zip_code_prefix,
                                    olist_geolocation_grouped.geolocation_lat,
                                    olist_geolocation_grouped.geolocation_lng):
            zip_diff = abs(zip_code - customer_zip)
            if zip_diff < zip_final_diff:
                zip_final_diff = zip_diff
                customer_lat = lat
                customer_lng = lng
                f_zip = zip_code
        df['customer_lat'] = customer_lat
        df['customer_lng'] = customer_lng  
        #Uncomment to check function results
        #print(f'customer-> {customer_zip}-{f_zip} diff ={zip_final_diff}')
        
    #fixing seller coordinates
    if mt.isnan(df.seller_lat):
        seller_zip = df.seller_zip_code_prefix
        zip_final_diff = float('inf')
        for zip_code,lat,lng in zip(olist_geolocation_grouped.geolocation_zip_code_prefix,
                                    olist_geolocation_grouped.geolocation_lat,
                                    olist_geolocation_grouped.geolocation_lng):
            zip_diff = abs(zip_code - seller_zip)
            if zip_diff < zip_final_diff:
                zip_final_diff = zip_diff
                seller_lat = lat
                seller_lng = lng
                f_zip = zip_code
        df['seller_lat'] = seller_lat
        df['seller_lng'] = seller_lng
        #Uncomment to check function results
        #print(f'seller-> {seller_zip}-{f_zip} diff ={zip_final_diff}')
        
    return df


In [15]:
raw_freight_df2 = raw_freight_df.copy()

raw_freight_df2 = raw_freight_df2.apply(find_closest_coordinate,axis = 1)
raw_freight_df2.sample(3)

,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state,customer_lat,customer_lng,seller_lat,seller_lng
14415,d11c6e3675c8fa2f8cd707a48969429a,f8328e9125f089a968fc6b36116c4bb9,218d46b86c1881d022bce9c68a7d4b15,2017-11-22 13:16:01,112.00,16.54,4a84ad2704fc9497804fdf6f5b2af2d5,delivered,575.0,20.0,16.0,16.0,29090,ES,14070,SP,-20.258921,-40.267568,-21.112713,-47.790810
55880,bebfce59cea8500ade1ddffe679b422c,20f160a9cf2e856ee5eba259e9ae33f9,17e34d8224d27a541263c4c64b11a56b,2017-06-26 10:45:09,167.92,12.68,01271eb2284947f2e2abbe53fa15730a,delivered,556.0,21.0,16.0,16.0,13690,SP,14085,SP,-21.908634,-47.619584,-21.165898,-47.797969
71732,a16c1e5c3c877e414d0e584e18ccb860,c9c6fde711572c1ad99ca12728c6af00,562fc2f2c2863ab7e79a9e4388a58a14,2018-04-24 00:30:58,29.99,18.23,02ee17ed5705b0015ca334b76b8373e5,delivered,150.0,17.0,8.0,14.0,75660,GO,13070,SP,-18.134504,-49.041262,-22.892058,-47.082443


In [16]:
raw_freight_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   order_id                  112650 non-null  object 
 1   product_id                112650 non-null  object 
 2   seller_id                 112650 non-null  object 
 3   shipping_limit_date       112650 non-null  object 
 4   price                     112650 non-null  float64
 5   freight_value             112650 non-null  float64
 6   customer_id               112650 non-null  object 
 7   order_status              112650 non-null  object 
 8   product_weight_g          112632 non-null  float64
 9   product_length_cm         112632 non-null  float64
 10  product_height_cm         112632 non-null  float64
 11  product_width_cm          112632 non-null  float64
 12  customer_zip_code_prefix  112650 non-null  int64  
 13  customer_state            112650 non-null  o

We solved the missing coordinates!

Now we only have to deal with the 18 items without infos about weight and measurements. 

First of all, lets analyze what products are missing those infos

In [20]:
missing_measurements =  raw_freight_df2[pd.isnull(raw_freight_df2.product_weight_g)]
                                                  
print(f'shape: {missing_measurements.shape}'
      f'\nUnique products: {missing_measurements.product_id.nunique()}')
missing_measurements.head(2)

shape: (18, 20)
Unique products: 2


,order_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state,customer_lat,customer_lng,seller_lat,seller_lng
47595,101157d4fae1c9fb74a00a5dee265c25,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,2017-04-11 08:02:26,29.0,14.52,f72b2f8d9295ef93fd40a4c49f67a42b,delivered,NaN,NaN,NaN,NaN,22631,RJ,12327,SP,-23.003657,-43.340759,-23.302318,-45.971888
47596,1521c6bb7b1028154c8c67cf80fa809f,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,2017-04-07 10:10:16,29.0,16.05,ca29b2bf57243228e98eab2dab805ae9,delivered,NaN,NaN,NaN,NaN,73105,DF,12327,SP,-15.662755,-47.860392,-23.302318,-45.971888


There are only two products (and 18 orders) without measurements infos

Lets take a closer look at these products

In [22]:
olist_products_dataset[olist_products_dataset.product_id.isin(missing_measurements.product_id.unique().tolist())]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.0,865.0,3.0,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looks like one of the products has no info at all, and we don't have enough info to treat the other one. Considering that we have only 18 items in that condition, we will just drop these values. 

In [23]:
raw_freight_df2.dropna(inplace =True)
raw_freight_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112632 entries, 0 to 112649
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   order_id                  112632 non-null  object 
 1   product_id                112632 non-null  object 
 2   seller_id                 112632 non-null  object 
 3   shipping_limit_date       112632 non-null  object 
 4   price                     112632 non-null  float64
 5   freight_value             112632 non-null  float64
 6   customer_id               112632 non-null  object 
 7   order_status              112632 non-null  object 
 8   product_weight_g          112632 non-null  float64
 9   product_length_cm         112632 non-null  float64
 10  product_height_cm         112632 non-null  float64
 11  product_width_cm          112632 non-null  float64
 12  customer_zip_code_prefix  112632 non-null  int64  
 13  customer_state            112632 non-null  o

To finalize our treatment, we want to deal only with orders that have been delivered

In [27]:
print(f'possible order status: {raw_freight_df2.order_status.unique()}')

raw_freight_df3 = raw_freight_df2[raw_freight_df2.order_status == 'delivered']
raw_freight_df3.reset_index(drop = True, inplace = True)

print(f'\nFinal len: {len(raw_freight_df3)}')

possible order status: ['delivered' 'shipped' 'invoiced' 'canceled' 'processing' 'unavailable'
 'approved']

Final len: 110179


Now that our DataFrame is merged and treated, we can clean it a little bit.

We wont need all those key values. Also, we already filtered order_status, and since we have the latitudes and longitudes, we wont need the zip codes neither. 

Lets drop those columns and reorganize our data

In [30]:
raw_freight_df4 = raw_freight_df3.copy()

raw_freight_df4 = raw_freight_df4.loc[:,['freight_value','price'] + raw_freight_df.columns[7:].tolist()]
raw_freight_df4.drop(['customer_zip_code_prefix','seller_zip_code_prefix','order_status'],axis = 1,inplace = True)

print(f'Final Shape: {raw_freight_df4.shape}')
raw_freight_df4.head(3)

Final Shape: (110179, 12)


,freight_value,price,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_state,seller_state,customer_lat,customer_lng,seller_lat,seller_lng
0,13.29,58.9,650.0,28.0,9.0,14.0,RJ,SP,-21.762775,-41.309633,-22.496953,-44.127492
1,17.96,55.9,650.0,28.0,9.0,14.0,GO,SP,-17.884337,-51.716990,-22.496953,-44.127492
2,18.33,64.9,650.0,28.0,9.0,14.0,MG,SP,-19.914323,-43.975703,-22.496953,-44.127492


In [31]:
#lets save the df

raw_freight_df4.to_csv('olist_freight_base_df.csv',index = False)